# Задание
Возьмите датасет www.kaggle.com...ta/kernels

1. Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)
2. Обучите на нем модели ResNet 18 и VGG 16 с использованием Fine-tuning (5-10 эпох)
3. Добавьте аугментацию данных к пункту 2
4. Сравните качество всех 3 полученных подходов

**Задание со звездочкой**

Примените Fine-tuning ResNet 18 к Fashion Mnist. Удалось ли увидеть резкое увеличение качества?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torchvision as tv
import time

In [ ]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.12.0+cu113  Device: cuda


In [ ]:
BATCH_SIZE = 32

In [ ]:
transformer = tv.transforms.Compose([tv.transforms.Resize((255, 255)), tv.transforms.ToTensor()])

In [ ]:
train_dataset = tv.datasets.ImageFolder("/content/drive/MyDrive/HW/Deep learning/Modern convolutional architectures/hymenoptera_data/train", transform=transformer)
test_dataset = tv.datasets.ImageFolder("/content/drive/MyDrive/HW/Deep learning/Modern convolutional architectures/hymenoptera_data/val", transform=transformer)

train_iter = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_iter = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = torch.Tensor([0]), 0
    net.eval()
    for X, y in data_iter:
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [ ]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net)
        print('Epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

### ResNet 18 & VGG 16

==================================================

ResNet

In [ ]:
net_res = models.resnet18(pretrained = False)
net_res

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
net_res.fc

Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
net_res.fc = nn.Linear(in_features=512, out_features=2)

In [ ]:
print("Params to learn:")
params_to_update = []
for name,param in net_res.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [ ]:
lr, num_epochs = 0.0001, 5
trainer = torch.optim.Adam(net_res.parameters(), lr=lr)
train(net_res, train_iter, test_iter, trainer, num_epochs)

Step time since epoch: 11.043. Train acc: 0.688. Train Loss: 21.415
Step time since epoch: 19.039. Train acc: 0.781. Train Loss: 17.235
Step time since epoch: 27.374. Train acc: 1.000. Train Loss: 13.182
Step time since epoch: 38.639. Train acc: 0.844. Train Loss: 15.178
Step time since epoch: 66.518. Train acc: 0.000. Train Loss: 47.814
Step time since epoch: 94.611. Train acc: 0.000. Train Loss: 48.487
Step time since epoch: 122.839. Train acc: 0.000. Train Loss: 45.891
Step time since epoch: 140.760. Train acc: 0.000. Train Loss: 25.340
Epoch 1, loss 0.9612, train acc 0.434, test acc 0.458, time 254.1 sec
Step time since epoch: 9.001. Train acc: 1.000. Train Loss: 12.424
Step time since epoch: 16.534. Train acc: 1.000. Train Loss: 4.190
Step time since epoch: 23.964. Train acc: 1.000. Train Loss: 1.152
Step time since epoch: 31.447. Train acc: 0.844. Train Loss: 23.766
Step time since epoch: 38.946. Train acc: 0.000. Train Loss: 140.805
Step time since epoch: 46.397. Train acc: 0.00

=========================================

VGG

In [ ]:
net_vgg = models.vgg16(pretrained = False)
net_vgg

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
net_vgg.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
net_vgg.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=2048, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=2048, out_features=2048, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=2048, out_features=2, bias=True))

In [ ]:
print("Params to learn:")
params_to_update = []
for name,param in net_vgg.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 features.0.weight
	 features.0.bias
	 features.2.weight
	 features.2.bias
	 features.5.weight
	 features.5.bias
	 features.7.weight
	 features.7.bias
	 features.10.weight
	 features.10.bias
	 features.12.weight
	 features.12.bias
	 features.14.weight
	 features.14.bias
	 features.17.weight
	 features.17.bias
	 features.19.weight
	 features.19.bias
	 features.21.weight
	 features.21.bias
	 features.24.weight
	 features.24.bias
	 features.26.weight
	 features.26.bias
	 features.28.weight
	 features.28.bias
	 classifier.0.weight
	 classifier.0.bias
	 classifier.3.weight
	 classifier.3.bias
	 classifier.6.weight
	 classifier.6.bias


In [ ]:
lr, num_epochs = 0.0001, 5
trainer = torch.optim.Adam(net_vgg.parameters(), lr=lr)
train(net_vgg, train_iter, test_iter, trainer, num_epochs)

Step time since epoch: 55.285. Train acc: 0.781. Train Loss: 22.031
Step time since epoch: 110.580. Train acc: 1.000. Train Loss: 15.574
Step time since epoch: 163.972. Train acc: 1.000. Train Loss: 1.909
Step time since epoch: 219.062. Train acc: 0.844. Train Loss: 71.032
Step time since epoch: 272.428. Train acc: 0.000. Train Loss: 237.651
Step time since epoch: 328.187. Train acc: 0.000. Train Loss: 85.471
Step time since epoch: 382.610. Train acc: 0.000. Train Loss: 34.301
Step time since epoch: 418.537. Train acc: 0.000. Train Loss: 15.584
Epoch 1, loss 1.9818, train acc 0.475, test acc 0.458, time 501.0 sec
Step time since epoch: 55.675. Train acc: 1.000. Train Loss: 21.379
Step time since epoch: 109.128. Train acc: 1.000. Train Loss: 21.802
Step time since epoch: 163.711. Train acc: 1.000. Train Loss: 21.874
Step time since epoch: 216.911. Train acc: 0.844. Train Loss: 21.947
Step time since epoch: 271.594. Train acc: 0.000. Train Loss: 22.637
Step time since epoch: 324.702. Tra

## ResNet 18 & VGG 16 with Fine-tuning

==================================

ResNet 18

In [ ]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224,224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.ImageFolder("/content/drive/MyDrive/HW/Deep learning/Modern convolutional architectures/hymenoptera_data/train", transform=transoforms)
test_dataset = tv.datasets.ImageFolder("/content/drive/MyDrive/HW/Deep learning/Modern convolutional architectures/hymenoptera_data/val", transform=transoforms)

train_iter = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_iter = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
net_res_pt = models.resnet18(pretrained = True)
net_res_pt

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for param in net_res_pt.parameters():
    param.requires_grad = False

In [ ]:
net_res_pt.fc = nn.Linear(in_features=512, out_features=2)

In [ ]:
print("Params to learn:")
params_to_update = []
for name,param in net_res_pt.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
lr, num_epochs = 0.0001, 5
trainer = torch.optim.Adam(net_res_pt.parameters(), lr=lr)
train(net_res_pt, train_iter, test_iter, trainer, num_epochs)

Step time since epoch: 2.798. Train acc: 0.781. Train Loss: 18.378
Step time since epoch: 5.425. Train acc: 0.844. Train Loss: 17.243
Step time since epoch: 7.888. Train acc: 0.844. Train Loss: 15.991
Step time since epoch: 10.381. Train acc: 0.844. Train Loss: 15.635
Step time since epoch: 12.792. Train acc: 0.125. Train Loss: 36.579
Step time since epoch: 15.419. Train acc: 0.156. Train Loss: 37.116
Step time since epoch: 18.036. Train acc: 0.094. Train Loss: 36.992
Step time since epoch: 19.627. Train acc: 0.000. Train Loss: 22.459
Epoch 1, loss 0.8213, train acc 0.484, test acc 0.536, time 30.4 sec
Step time since epoch: 2.295. Train acc: 0.781. Train Loss: 17.154
Step time since epoch: 4.594. Train acc: 0.656. Train Loss: 19.298
Step time since epoch: 6.966. Train acc: 0.781. Train Loss: 19.902
Step time since epoch: 9.393. Train acc: 0.656. Train Loss: 17.995
Step time since epoch: 11.695. Train acc: 0.312. Train Loss: 26.996
Step time since epoch: 13.947. Train acc: 0.500. Train

==============================

VGG_16

In [ ]:
net_vgg_pt = models.vgg16(pretrained = True)
net_vgg_pt

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
for param in net_vgg_pt.parameters():
    param.requires_grad = False

In [ ]:
net_vgg_pt.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=2048, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=2048, out_features=2048, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=2048, out_features=2, bias=True))

In [ ]:
print("Params to learn:")
params_to_update = []
for name,param in net_res_pt.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
lr, num_epochs = 0.0001, 5
trainer = torch.optim.Adam(net_vgg_pt.parameters(), lr=lr)
train(net_vgg_pt, train_iter, test_iter, trainer, num_epochs)

Step time since epoch: 14.508. Train acc: 0.688. Train Loss: 21.551
Step time since epoch: 28.970. Train acc: 1.000. Train Loss: 6.412
Step time since epoch: 43.169. Train acc: 1.000. Train Loss: 1.210
Step time since epoch: 57.424. Train acc: 0.844. Train Loss: 32.833
Step time since epoch: 71.701. Train acc: 0.000. Train Loss: 235.850
Step time since epoch: 85.978. Train acc: 0.000. Train Loss: 157.167
Step time since epoch: 101.657. Train acc: 0.000. Train Loss: 112.368
Step time since epoch: 110.790. Train acc: 0.000. Train Loss: 40.603
Epoch 1, loss 2.4918, train acc 0.463, test acc 0.458, time 176.2 sec
Step time since epoch: 14.134. Train acc: 1.000. Train Loss: 9.665
Step time since epoch: 28.430. Train acc: 1.000. Train Loss: 16.079
Step time since epoch: 44.184. Train acc: 1.000. Train Loss: 19.851
Step time since epoch: 58.612. Train acc: 0.906. Train Loss: 20.580
Step time since epoch: 72.691. Train acc: 0.000. Train Loss: 23.645
Step time since epoch: 86.810. Train acc: 0.

## Augmintation

In [ ]:
trasform_train = tv.transforms.Compose([
        tv.transforms.RandomResizedCrop(224),
        tv.transforms.RandomHorizontalFlip(),
        tv.transforms.RandomVerticalFlip(),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

trasform_test = tv.transforms.Compose([
        tv.transforms.Resize(256),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

train_dataset_aug = tv.datasets.ImageFolder("/content/drive/MyDrive/HW/Deep learning/Modern convolutional architectures/hymenoptera_data/train", transform=trasform_train)
test_dataset_aug = tv.datasets.ImageFolder("/content/drive/MyDrive/HW/Deep learning/Modern convolutional architectures/hymenoptera_data/val", transform=trasform_test)

train_iter_aug = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_iter_aug = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

ResNet 18 + aug

In [ ]:
lr, num_epochs = 0.0001, 5
trainer = torch.optim.Adam(net_res_pt.parameters(), lr=lr)
train(net_res_pt, train_iter_aug, test_iter_aug, trainer, num_epochs)

Step time since epoch: 2.456. Train acc: 0.781. Train Loss: 18.533
Step time since epoch: 4.872. Train acc: 0.844. Train Loss: 17.421
Step time since epoch: 7.309. Train acc: 0.844. Train Loss: 16.151
Step time since epoch: 9.775. Train acc: 0.875. Train Loss: 15.403
Step time since epoch: 12.200. Train acc: 0.125. Train Loss: 36.354
Step time since epoch: 14.600. Train acc: 0.156. Train Loss: 36.832
Step time since epoch: 16.975. Train acc: 0.094. Train Loss: 36.714
Step time since epoch: 18.453. Train acc: 0.000. Train Loss: 22.320
Epoch 1, loss 0.8186, train acc 0.488, test acc 0.614, time 28.9 sec
Step time since epoch: 2.150. Train acc: 0.875. Train Loss: 15.005
Step time since epoch: 4.311. Train acc: 0.656. Train Loss: 17.966
Step time since epoch: 6.458. Train acc: 0.781. Train Loss: 18.012
Step time since epoch: 8.660. Train acc: 0.781. Train Loss: 15.422
Step time since epoch: 10.820. Train acc: 0.594. Train Loss: 24.931
Step time since epoch: 12.979. Train acc: 0.688. Train 

VGG 16 + aug

In [ ]:
lr, num_epochs = 0.0001, 5
trainer = torch.optim.Adam(net_res_pt.parameters(), lr=lr)
train(net_vgg_pt, train_iter_aug, test_iter_aug, trainer, num_epochs)

Step time since epoch: 14.095. Train acc: 0.938. Train Loss: 9.629
Step time since epoch: 28.312. Train acc: 0.969. Train Loss: 9.606
Step time since epoch: 42.341. Train acc: 0.875. Train Loss: 13.042
Step time since epoch: 56.252. Train acc: 0.938. Train Loss: 10.327
Step time since epoch: 69.973. Train acc: 1.000. Train Loss: 5.883
Step time since epoch: 85.259. Train acc: 1.000. Train Loss: 6.949
Step time since epoch: 99.180. Train acc: 1.000. Train Loss: 6.598
Step time since epoch: 107.804. Train acc: 0.950. Train Loss: 3.918
Epoch 1, loss 0.2703, train acc 0.959, test acc 0.876, time 173.8 sec
Step time since epoch: 13.763. Train acc: 0.938. Train Loss: 7.867
Step time since epoch: 28.468. Train acc: 0.969. Train Loss: 8.710
Step time since epoch: 42.882. Train acc: 0.938. Train Loss: 11.628
Step time since epoch: 56.836. Train acc: 0.969. Train Loss: 8.072
Step time since epoch: 70.695. Train acc: 1.000. Train Loss: 5.175
Step time since epoch: 84.706. Train acc: 0.969. Train 

Лучшее значение показала предобученая VGG с ft

## Fine-tuning ResNet 18 in Fashion Mnist

In [ ]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224,224)),
    tv.transforms.ToTensor()
])

train_dataset_mnist = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset_mnist = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)

train_iter_mnist = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter_mnist = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
model = tv.models.resnet18(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
model.fc = nn.Linear(in_features=512, out_features=10)

In [ ]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
lr, num_epochs = 0.0001, 5
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, train_iter_mnist, test_iter_mnist, trainer, num_epochs)

Step time since epoch: 2.461. Train acc: 0.000. Train Loss: 78.872
Step time since epoch: 4.919. Train acc: 0.000. Train Loss: 75.522
Step time since epoch: 7.375. Train acc: 0.031. Train Loss: 72.823
Step time since epoch: 9.926. Train acc: 0.031. Train Loss: 69.224
Step time since epoch: 12.470. Train acc: 0.156. Train Loss: 67.159
Step time since epoch: 14.951. Train acc: 0.125. Train Loss: 66.152
Step time since epoch: 17.414. Train acc: 0.062. Train Loss: 63.633
Step time since epoch: 18.944. Train acc: 0.100. Train Loss: 38.697
Epoch 1, loss 2.1807, train acc 0.061, test acc 0.333, time 29.9 sec
Step time since epoch: 2.220. Train acc: 0.531. Train Loss: 49.096
Step time since epoch: 4.463. Train acc: 0.531. Train Loss: 47.510
Step time since epoch: 6.709. Train acc: 0.594. Train Loss: 48.869
Step time since epoch: 8.998. Train acc: 0.594. Train Loss: 46.811
Step time since epoch: 11.274. Train acc: 0.406. Train Loss: 50.033
Step time since epoch: 13.578. Train acc: 0.375. Train 

Значение сильно просело, тогда CNN показал 90% 